In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV, Ridge
from sklearn.model_selection import ParameterGrid, ParameterSampler, RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, KernelCenterer, RobustScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer, max_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib
from sklearn.model_selection import cross_val_score, KFold, cross_validate, cross_val_predict
from sklearn.pipeline import make_pipeline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, time
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, max_error
from sklearn.linear_model import LinearRegression

## BEFORE PRUNING

In [5]:
df_all = pd.read_csv('data_all.csv')
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [6]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    ('random_forest', RandomForestRegressor(n_estimators=250, random_state=42, max_features = 'sqrt')),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=200, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999518449048 
 Beta : 0.9786085926247874 
 V : 0.9999997006737554 
MAE : 
 Alpha : 0.004298451266537389 
 Beta : 0.550168122812585 
 V : 0.0021246872693424417 
MSE : 
 Alpha : 5.760326756291966e-05 
 Beta : 2.251416282746785 
 V : 6.128302128893325e-05 
Max AERR : 
 V     0.081751
α     0.075188
β    12.733468
dtype: float64
Waktu yang dibutuhkan :  1.484109310309092


In [91]:
df_all_before = pd.read_csv('data_all.csv')
df_test_before = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 0) & (df_all_before["β"] <= 0) ]
print(len(df_test_before))
# Alpha Test Data
df_test_alpha = df_test_before.copy()
df_test_alpha["α"] *= -1 
df_test_alpha[['Pα1', 'Pα2']] = df_test_alpha[['Pα2', 'Pα1']]
df_test_alpha
print(len(df_test_alpha))

# Beta Test Data
df_test_beta = df_test_before.copy()
df_test_beta["β"] *= -1 
df_test_beta[['Pβ1', 'Pβ2']] = df_test_beta[['Pβ2', 'Pβ1']]
df_test_beta
print(len(df_test_beta))

# Combine Test Data
df_test_alpha_predict = df_test_alpha.sample(n = 500, random_state = 42)
df_test_beta_predict = df_test_beta.sample(n = 500, random_state = 42)
df_test_combine = pd.concat([df_test_alpha_predict,df_test_beta_predict], ignore_index = True)
df_test_combine

# Separation Data Test
x_test = df_test_combine.drop(columns = ["V","α","β"])
y_test = df_test_combine[["V","α","β"]]


1300
1300
1300


In [92]:
# Evaluate the metrics
y0_test_pred = model_all_best.predict(x_test)
r2_test = r2_score(y_test, y0_test_pred, multioutput = 'raw_values')
mae_test = mean_absolute_error(y_test, y0_test_pred, multioutput = 'raw_values')
mse_test = mean_squared_error(y_test, y0_test_pred, multioutput = 'raw_values')
max_ae_test = max_absolute_error(y_test, y0_test_pred)
print(f'R2 : \n Alpha : {r2_test[1]} \n Beta : {r2_test[2]} \n V : {r2_test[0]} ')
print(f'MAE : \n Alpha : {mae_test[1]} \n Beta : {mae_test[2]} \n V : {mae_test[0]} ')
print(f'MSE : \n Alpha : {mse_test[1]} \n Beta : {mse_test[2]} \n V : {mse_test[0]} ')
print('Max AERR : \n', max_ae_test)

R2 : 
 Alpha : 0.9991078213038361 
 Beta : 0.9911652936965382 
 V : 0.9999868415142047 
MAE : 
 Alpha : 0.07841954608057887 
 Beta : 0.6978142029924486 
 V : 0.03182893363224958 
MSE : 
 Alpha : 0.22893305343564424 
 Beta : 1.5332632789658036 
 V : 0.002689702609532636 
Max AERR : 
 V    0.313002
α    4.942151
β    6.199186
dtype: float64


## AFTER PRUNING

In [93]:
df_all = pd.read_csv('data_clean_reduce_24.csv')
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [94]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=200, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999888364483 
 Beta : 0.9855572270835813 
 V : 0.9999998576576354 
MAE : 
 Alpha : 0.0018578545045363193 
 Beta : 0.4649694558260702 
 V : 0.0014418516960277908 
MSE : 
 Alpha : 1.2746920367403166e-05 
 Beta : 1.5371880960149589 
 V : 2.9044107812892754e-05 
Max AERR : 
 V     0.050330
α     0.048791
β    10.644980
dtype: float64
Waktu yang dibutuhkan :  1.4505133827527363


In [95]:
df_all_before = pd.read_csv('data_clean_reduce_24.csv')
df_test_before = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 0) & (df_all_before["β"] <= 0) ]
print(len(df_test_before))
# Alpha Test Data
df_test_alpha = df_test_before.copy()
df_test_alpha["α"] *= -1 
df_test_alpha[['Pα1', 'Pα2']] = df_test_alpha[['Pα2', 'Pα1']]
df_test_alpha
print(len(df_test_alpha))

# Beta Test Data
df_test_beta = df_test_before.copy()
df_test_beta["β"] *= -1 
df_test_beta[['Pβ1', 'Pβ2']] = df_test_beta[['Pβ2', 'Pβ1']]
df_test_beta
print(len(df_test_beta))

# Combine Test Data
df_test_alpha_predict = df_test_alpha.sample(n = 500, random_state = 42)
df_test_beta_predict = df_test_beta.sample(n = 500, random_state = 42)
df_test_combine = pd.concat([df_test_alpha_predict,df_test_beta_predict], ignore_index = True)
df_test_combine

# Separation Data Test
x_test = df_test_combine.drop(columns = ["V","α","β"])
y_test = df_test_combine[["V","α","β"]]


1300
1300
1300


In [96]:
# Evaluate the metrics
y0_test_pred = model_all_best.predict(x_test)
r2_test = r2_score(y_test, y0_test_pred, multioutput = 'raw_values')
mae_test = mean_absolute_error(y_test, y0_test_pred, multioutput = 'raw_values')
mse_test = mean_squared_error(y_test, y0_test_pred, multioutput = 'raw_values')
max_ae_test = max_absolute_error(y_test, y0_test_pred)
print(f'R2 : \n Alpha : {r2_test[1]} \n Beta : {r2_test[2]} \n V : {r2_test[0]} ')
print(f'MAE : \n Alpha : {mae_test[1]} \n Beta : {mae_test[2]} \n V : {mae_test[0]} ')
print(f'MSE : \n Alpha : {mse_test[1]} \n Beta : {mse_test[2]} \n V : {mse_test[0]} ')
print('Max AERR : \n', max_ae_test)

R2 : 
 Alpha : 0.9989988491562861 
 Beta : 0.9922479265670952 
 V : 0.9999868458478064 
MAE : 
 Alpha : 0.07667837992395815 
 Beta : 0.6625202529467438 
 V : 0.03179980184788013 
MSE : 
 Alpha : 0.2568953064969703 
 Beta : 1.3453723442806427 
 V : 0.002688816785736036 
Max AERR : 
 V    0.312596
α    4.978439
β    5.908332
dtype: float64
